In [1]:
import bagpy
from bagpy import bagreader
from datetime import datetime
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import seaborn as sns
import logging
import re
import os

In [67]:
PID = '6_EXP04/'
logging.basicConfig(level=logging.DEBUG, filename=PID + 'log', filemode="a+", format="%(asctime)-15s %(levelname)-8s %(message)s")

In [27]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def get_timestamp(file):
    start_year = find_nth(file, '_', 2) + 1
    start_month = find_nth(file, '-', 1) + 1
    start_day = find_nth(file, '-', 2) + 1
    start_hour = find_nth(file, '-', 3) + 1
    start_minute = find_nth(file, '-', 4) + 1
    start_second = find_nth(file, '-', 5) + 1
    
    end_year = find_nth(file, '-', 1)
    end_month = find_nth(file, '-', 2)
    end_day = find_nth(file, '-', 3)
    end_hour = find_nth(file, '-', 4)
    end_minute = find_nth(file, '-', 5)
    end_second = find_nth(file, '_', 3)
    
    year = file[start_year:end_year]
    month = file[start_month:end_month]
    day = file[start_day:end_day]
    hour = file[start_hour:end_hour]
    minute = file[start_minute:end_minute]
    second = file[start_second:end_second]
    
    return year, month, day, hour, minute, second

# y, mo, d, h, mi, s = get_timestamp(rosbag)

In [28]:
def read_bags(source):
    bags = [f for f in os.listdir(source) if f.endswith('.bag')]

    df_attention = None
    df_robotgaze = None
    df_robotbehave = None

    for bag in bags:
        try:
            b = bagreader(source + bag)

            b_attention = b.message_by_topic('/sar/perception/attention_targets')
            b_robotgaze = b.message_by_topic('/sar/jibo/robot_attention_target')
            b_robotbehave = b.message_by_topic('/sar/jibo/verbose_state')

            temp_attention = pd.read_csv(b_attention)
            temp_robotgaze = pd.read_csv(b_robotgaze)
            temp_robotbehave = pd.read_csv(b_robotbehave)

            df_attention = pd.concat([df_attention, temp_attention], ignore_index=True)
            df_robotgaze = pd.concat([df_robotgaze, temp_robotgaze], ignore_index=True)
            df_robotbehave = pd.concat([df_robotbehave, temp_robotbehave], ignore_index=True)
        
        except Exception as e:
            logging.error('Error occurred on ' + bag + ': ' + str(e))
    
    return df_attention, df_robotgaze, df_robotbehave

In [68]:
df_attention_w1, df_robotgaze_w1, df_robotbehave_w1 = read_bags(PID + 'week_1/')
df_attention_w2, df_robotgaze_w2, df_robotbehave_w2 = read_bags(PID + 'week_2/')
df_attention_w3, df_robotgaze_w3, df_robotbehave_w3 = read_bags(PID + 'week_3/')
df_attention_w4, df_robotgaze_w4, df_robotbehave_w4 = read_bags(PID + 'week_4/')

[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-28-19-08-16_0.
[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-28-19-18-16_1.
[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-28-19-28-16_2.
[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-28-19-38-16_3.
No data on the topic:/sar/jibo/verbose_state
[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-29-17-56-17_0.
No data on the topic:/sar/perception/attention_targets
[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-29-18-04-12_0.
No data on the topic:/sar/perception/attention_targets
No data on the topic:/sar/jibo/robot_attention_target
No data on the topic:/sar/jibo/verbose_state
[INFO]  Successfully created the data folder 6_EXP04/week_1/rosbag_common_2017-08-29-18-09-45_0.
No data on the topic:/sar/perception/attention_targets
[INFO]  Succ

## Gaze Tracking

In [30]:
def get_region(role, attention_string):
    attention = re.sub(r'\W+', '', attention_string)
    start_region = find_nth(attention, role, 1) + len(role) + len('region')
    end_region = find_nth(attention[start_region:], 'role', 1) + start_region
    end_region = len(attention) if (end_region == -1 or end_region < start_region) else end_region
    region = attention[start_region:end_region]
    return region

In [31]:
def get_human_attention(df, role):

    prev_object = get_region(role, df['attention'].at[0])
    prev_timestamp = float(df['Time'].at[0])
    duration = 0
    start_index = 0

    role_attention = []
    role_timestamp = []
    role_duration = []

    for index, row in df.iterrows():
        curr_object = get_region(role, row['attention'])

        # same attentional object
        if curr_object == prev_object:
            duration = duration + (float(row['Time']) - prev_timestamp)
            prev_timestamp = float(row['Time'])

        # detected change in attentional object
        else:
            # record the object and duration
            role_attention.append(prev_object)
            role_duration.append(duration)
            role_timestamp.append(df['Time'].at[start_index])

            # reset pointers
            prev_object = curr_object
            prev_timestamp = float(row['Time'])

            # reset duration for new object
            duration = 0
            start_index = index

            # print(role_attention[-1], role_duration[-1], role_timestamp[-1])
    
    return role_attention, role_timestamp, role_duration

In [32]:
def get_robot_attention(df):

    prev_object = df['data'].at[0]
    prev_timestamp = float(df['Time'].at[0])
    duration = 0
    start_index = 0

    role_attention = []
    role_timestamp = []
    role_duration = []

    for index, row in df.iterrows():
        curr_object = row['data']

        # same attentional object
        if curr_object == prev_object:
            duration = duration + (float(row['Time']) - prev_timestamp)
            prev_timestamp = float(row['Time'])

        # detected change in attentional object
        else:
            # record the object and duration
            role_attention.append(prev_object)
            role_duration.append(duration)
            role_timestamp.append(df['Time'].at[start_index])

            # reset pointers
            prev_object = curr_object
            prev_timestamp = float(row['Time'])

            # reset duration for new object
            duration = 0
            start_index = index

            # print(role_attention[-1], role_duration[-1], role_timestamp[-1])
    
    return role_attention, role_timestamp, role_duration

In [33]:
def attention_csvs(source, df_attention, df_robotgaze):
    c_objects, c_timestamps, c_durations = get_human_attention(df_attention, 'child')
    p_objects, p_timestamps, p_durations = get_human_attention(df_attention, 'parent')
    r_objects, r_timestamps, r_durations = get_robot_attention(df_robotgaze)
    
    child = pd.DataFrame({'object': c_objects, 'duration': c_durations, 'start': c_timestamps})
    parent = pd.DataFrame({'object': p_objects, 'duration': p_durations, 'start': p_timestamps})
    robot = pd.DataFrame({'object': r_objects, 'duration': r_durations, 'start': r_timestamps})
    
    child.to_csv(source + 'child.csv')
    parent.to_csv(source + 'parent.csv')
    robot.to_csv(source + 'robot.csv')
    
    return child, parent, robot

In [69]:
df_c_w1, df_p_w1, df_r_w1 = attention_csvs(PID + 'week_1/', df_attention_w1, df_robotgaze_w1)
df_c_w2, df_p_w2, df_r_w2 = attention_csvs(PID + 'week_2/', df_attention_w2, df_robotgaze_w2)
df_c_w3, df_p_w3, df_r_w3 = attention_csvs(PID + 'week_3/', df_attention_w3, df_robotgaze_w3)
df_c_w4, df_p_w4, df_r_w4 = attention_csvs(PID + 'week_4/', df_attention_w4, df_robotgaze_w4)

In [82]:
# all participants
PIDS = ['5_EXP07/']

In [83]:
for p in PIDS:
    print('processing', p)
    
    df_c_w1, df_p_w1, df_r_w1 = attention_csvs(p + 'week_1/', df_attention_w1, df_robotgaze_w1)
    df_c_w2, df_p_w2, df_r_w2 = attention_csvs(p + 'week_2/', df_attention_w2, df_robotgaze_w2)
    df_c_w3, df_p_w3, df_r_w3 = attention_csvs(p + 'week_3/', df_attention_w3, df_robotgaze_w3)
    df_c_w4, df_p_w4, df_r_w4 = attention_csvs(p + 'week_4/', df_attention_w4, df_robotgaze_w4)

processing 5_EXP07/


## Joint Attention

In [35]:
def get_overlap(df1, df2, self):
    
    df1.name = self[0]
    df2.name = self[1]
    
    # determine the object with the more frequent changes in gaze
    primary = None
    secondary = None
    if len(df1) > len(df2):
        primary = df1
        secondary = df2
    else:
        primary = df2
        secondary = df1
        
    joint_object = []
    joint_duration = []
    joint_timestamp = []
    joint_initiator = [] # who looked first?
    joint_lag = [] # response lag
    
    for index_p, row_p in primary.iterrows():
        for index_s, row_s in secondary.iterrows():
            if (
                (row_s['object'] == row_p['object']) or # looking at the same object
                (row_s['object'] in self) or # object 1 is looking at object 2
                (row_p['object'] in self) # object 2 is looking at object 1
            ) and ( # timestamps overlap
                float(row_s['start']) <= float(row_p['start']) <= (float(row_s['start']) + float(row_s['duration']))
            ):
                start = max([float(row_p['start']), float(row_s['start'])])
                duration = min([float(row_p['duration']), float(row_s['duration'])])
                initiator = secondary.name if row_s['start'] == min([float(row_p['start']), float(row_s['start'])]) else primary.name
                
                joint_object.append('contact' if row_s['object'] in self else row_s['object'])
                joint_timestamp.append(start)
                joint_duration.append(duration)
                joint_initiator.append(initiator)
                joint_lag.append(start - min([float(row_p['start']), float(row_s['start'])]))
                
                # print(joint_object[-1], joint_duration[-1], joint_timestamp[-1], joint_initiator[-1], joint_lag[-1])
                break

    return joint_object, joint_timestamp, joint_duration, joint_initiator, joint_lag

In [36]:
def joint_attention_csvs(source, child, parent, robot):
    cr_objects, cr_timestamps, cr_durations, cr_initiator, cr_lag = get_overlap(child, robot, ['child', 'robot'])
    cp_objects, cp_timestamps, cp_durations, cp_initiator, cp_lag = get_overlap(child, parent, ['child', 'parent'])
    pr_objects, pr_timestamps, pr_durations, pr_initiator, pr_lag = get_overlap(parent, robot, ['parent', 'robot'])
    
    child_robot = pd.DataFrame({'object': cr_objects, 'duration': cr_durations, 'start': cr_timestamps, 'initiator': cr_initiator, 'lag': cr_lag})
    child_parent = pd.DataFrame({'object': cp_objects, 'duration': cp_durations, 'start': cp_timestamps, 'initiator': cp_initiator, 'lag': cp_lag})
    parent_robot = pd.DataFrame({'object': pr_objects, 'duration': pr_durations, 'start': pr_timestamps, 'initiator': pr_initiator, 'lag': pr_lag})

    child_robot.to_csv(source + 'child-robot.csv', index=False)
    child_parent.to_csv(source + 'child-parent.csv', index=False)
    parent_robot.to_csv(source + 'parent-robot.csv', index=False)
    
    return child_robot, child_parent, parent_robot

In [71]:
df_cr_w1, df_cp_w1, df_pr_w1 = joint_attention_csvs(PID + 'week_1/', df_c_w1, df_p_w1, df_r_w1)
df_cr_w2, df_cp_w2, df_pr_w2 = joint_attention_csvs(PID + 'week_2/', df_c_w2, df_p_w2, df_r_w2)
df_cr_w3, df_cp_w3, df_pr_w3 = joint_attention_csvs(PID + 'week_3/', df_c_w3, df_p_w3, df_r_w3)
df_cr_w4, df_cp_w4, df_pr_w4 = joint_attention_csvs(PID + 'week_4/', df_c_w4, df_p_w4, df_r_w4)

KeyboardInterrupt: 

## Event Stimulus

In [38]:
def get_speech_sentiment(s):
    sentence = s.lower()
    
    questions = ('what', 'will', 'who', 'where', 'why', 'when', 'how', 'can', 'may', 'could')
    positive_feedback = ('thank', 'super', 'nice', 'good job', 'great', 'perfect')
    negative_feedback = ('try', 'again', 'almost up')
    commands = ('finish', 'help', 'move', 'put', 'sort')
    
    if sentence.endswith('?') or sentence.startswith(questions) or ('please' in sentence):
        return 'prompt'
    elif any(item in sentence for item in positive_feedback):
        return 'positive feedback'
    elif any(item in sentence for item in negative_feedback):
        return 'negative feedback'
    elif any(item in sentence for item in commands):
        return 'command'
    else:
        return 'content'

In [39]:
def get_action_sentiment(action):
    positive_actions = ('excited', 'smile', 'laugh', 'happy', 'greeting', 'sing-song')
    
    if any(item in action for item in positive_actions):
        return 'positive'
    elif 'focus' in action:
        return 'directive'
    else:
        return 'content'

In [40]:
def get_robot_speech(df):
    prev_speech_timestamp = -1
    prev_action_timestamp = -1
    # prev_gaze_timestamp = -1
    
    prev_content = ""
    speech_timer = True
    action_timer = True
    # gaze_timer = 0
    
    category = []
    content = []
    timestamp = []
    duration = []
    sentiment = []
    
    speech_main_key = 'tts '
    speech_begin_key = ': started'
    speech_end_key = ': stopped'
    punctuation = ('?', '!', '.')
    
    action_main_key = 'animation '
    action_begin_keys = (': started', ': received')
    action_end_key = ': stopped'
    
    # gaze_main_key = 'face following '
    
    for index, row in df.iterrows():
        
        '''
        # gaze
        if gaze_main_key in row['data']:
            if gaze_timer == 0:
                prev_gaze_timestamp = float(row['Time'])
                gaze_timer = 1
            else:
                gaze_timer = gaze_timer + 1
        elif gaze_timer > 2:
            duration.append(float(row['Time']) - prev_gaze_timestamp)
            timestamp.append(prev_gaze_timestamp)
            content.append('face following')
            sentiment.append('n/a')
            category.append('gaze')
            gaze_timer = 0
        '''
            
        # speech
        if speech_main_key in row['data']:
            if speech_begin_key in row['data'] and speech_timer:
                prev_speech_timestamp = float(row['Time'])
                speech_timer = False
            elif speech_end_key in row['data']:
                prev_content = prev_content + row['data'][len(speech_main_key) + 1:(len(row['data']) - len(speech_end_key) - 1)]
                if prev_content.endswith(punctuation):
                    duration.append(float(row['Time']) - prev_speech_timestamp)
                    timestamp.append(prev_speech_timestamp)
                    content.append(prev_content)
                    sentiment.append(get_speech_sentiment(prev_content))
                    category.append('speech')
                    prev_content = ""
                    speech_timer = True

        # action
        elif action_main_key in row['data']:
            if any(key in row['data'] for key in action_begin_keys):
                prev_action_timestamp = float(row['Time'])
            elif action_end_key in row['data']:
                description = row['data'][(row['data'].find('[') + 1):(row['data'].find(']'))]
                duration.append(float(row['Time']) - prev_action_timestamp)
                timestamp.append(prev_action_timestamp)
                content.append(description)
                sentiment.append(get_action_sentiment(description))
                category.append('action')
    
    return category, content, sentiment, timestamp, duration

In [ ]:
r1_category, r1_content, r1_sentiment, r1_start, r1_duration = get_robot_speech(df_robotbehave_w1)
r2_category, r2_content, r2_sentiment, r2_start, r2_duration = get_robot_speech(df_robotbehave_w2)
r3_category, r3_content, r3_sentiment, r3_start, r3_duration = get_robot_speech(df_robotbehave_w3)
r4_category, r4_content, r4_sentiment, r4_start, r4_duration = get_robot_speech(df_robotbehave_w4)

In [ ]:
r1_behavior = pd.DataFrame({'category': r1_category, 'content': r1_content, 'sentiment': r1_sentiment, 'start': r1_start, 'duration': r1_duration})
r2_behavior = pd.DataFrame({'category': r2_category, 'content': r2_content, 'sentiment': r2_sentiment, 'start': r2_start, 'duration': r2_duration})
r3_behavior = pd.DataFrame({'category': r3_category, 'content': r3_content, 'sentiment': r3_sentiment, 'start': r3_start, 'duration': r3_duration})
r4_behavior = pd.DataFrame({'category': r4_category, 'content': r4_content, 'sentiment': r4_sentiment, 'start': r4_start, 'duration': r4_duration})

In [ ]:
r1_behavior.to_csv(PID + 'week_1/' + 'robot_behavior.csv', index=False)
r2_behavior.to_csv(PID + 'week_2/' + 'robot_behavior.csv', index=False)
r3_behavior.to_csv(PID + 'week_3/' + 'robot_behavior.csv', index=False)
r4_behavior.to_csv(PID + 'week_4/' + 'robot_behavior.csv', index=False)